In [ ]:
import kaggle

In [ ]:
!kaggle competitions download -c ds1-tree-ensembles

In [ ]:
!pwd

"age";"job";"marital";"education";"default";"housing";"loan";"contact";"month";"day_of_week";"duration";"campaign";"pdays";"previous";"poutcome";"emp.var.rate";"cons.price.idx";"cons.conf.idx";"euribor3m";"nr.employed";"y



In [193]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
import numpy as np
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from ipywidgets import interact
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.plotting import plot_decision_regions
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from mlxtend.plotting import plot_decision_regions
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [194]:
emps = ['nurse', 'doctor', 'lead', 'plumber', 'coal', 'accountant', 'attorney']
manager = ['manager','vp', 'president', 'vice president''director', 'executive', 'superintendent', 'captain']
it = ['tech','computers', 'it', 'data', 'progammer', 'analyst']

In [195]:
df = pd.read_csv('train_features.csv')

In [196]:
df.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,43373,NaN,12000,12000,36 months,16.02%,422.01,C,C5,Driver,4 years,MORTGAGE,81000.0,NaN,NaN,debt_consolidation,Debt consolidation,280xx,NC,12.76,0,Dec-2005,1,48.0,NaN,7,0,4912,23.5%,15,w,0,48.0,Individual,NaN,NaN,0,0,174783,1,2,0,4,13.0,33668,82.0,2,5,2962,63.0,20900,1,1,2,9,24969.0,588.0,89.3,0,0,145.0,23,1,1,1,8.0,NaN,1.0,NaN,2,2,2,2,2,9,4,5,2,7,0.0,0,0,2,92.9,100.0,0,0,206618,38580,5500,40863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,24414,NaN,6300,6300,36 months,14.07%,215.54,C,C3,GMP Lead,4 years,RENT,39000.0,NaN,NaN,debt_consolidation,Debt consolidation,920xx,CA,21.42,0,Mar-2012,1,NaN,NaN,10,0,11876,59.4%,12,w,0,NaN,Individual,NaN,NaN,0,0,31046,0,1,0,2,14.0,19170,83.0,3,8,5153,72.0,20000,1,0,1,10,3105.0,6877.0,59.3,0,0,17.0,72,8,8,0,8.0,NaN,6.0,NaN,0,6,7,7,8,2,9,10,7,10,0.0,0,0,3,100.0,42.9,0,0,43140,31046,16900,23140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,46723,NaN,4500,4500,36 months,7.21%,139.38,A,A3,Accounts,3 years,RENT,78000.0,NaN,NaN,debt_consolidation,Debt consolidation,906xx,CA,2.17,0,May-2000,2,NaN,NaN,13,0,1715,5.2%,19,w,0,NaN,Individual,NaN,NaN,0,0,35329,0,8,1,2,10.0,33614,106.0,1,1,1196,54.0,33300,4,0,3,3,2718.0,21585.0,7.4,0,0,166.0,218,10,10,0,31.0,NaN,1.0,NaN,0,3,3,3,4,13,5,6,3,13,0.0,0,0,2,100.0,0.0,0,0,65092,35329,23300,31792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,24878,NaN,12000,12000,36 months,9.44%,384.06,B,B1,Client Processing Associate,10+ years,MORTGAGE,62000.0,NaN,NaN,debt_consolidation,Debt consolidation,441xx,OH,13.76,0,Feb-2003,0,NaN,NaN,22,0,9404,16.3%,33,w,0,NaN,Individual,NaN,NaN,0,0,75160,4,1,0,2,13.0,17123,67.0,6,9,6898,32.0,57600,3,1,2,12,3579.0,33602.0,17.0,0,0,162.0,179,1,1,1,1.0,NaN,5.0,NaN,0,1,3,9,13,7,20,25,3,22,0.0,0,0,7,100.0,0.0,0,0,133065,26527,40500,25465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,39038,NaN,12000,12000,36 months,22.35%,460.47,D,D5,Construction Operator,10+ years,MORTGAGE,70000.0,NaN,NaN,debt_consolidation,Debt consolidation,751xx,TX,14.04,4,Nov-2001,2,13.0,NaN,18,0,17895,24.2%,45,w,0,13.0,Individual,NaN,NaN,0,7218,113702,9,2,2,4,6.0,16711,68.0,9,10,5373,35.0,73900,3,6,3,14,6317.0,32863.0,24.1,0,0,159.0,201,0,0,1,0.0,62.0,3.0,62.0,6,9,11,11,14,22,15,20,11,18,0.0,0,2,11,82.2,9.1,0,0,192842,34606,43300,24499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash


In [197]:
earliest_cr_line = df.earliest_cr_line.apply(lambda d: datetime.strptime(d,'%b-%Y'))
df_earliest_cr_line_min = earliest_cr_line.min()

In [198]:
means = {}
count_null_subgrade_and_grade = 0

In [199]:
def wrangle(X, make_means=False):
    X = X.copy()

    # Drop some columns
    X = X.drop(columns='id')  # id is random
    X = X.drop(columns=['member_id', 'url', 'desc'])  # All null
    X = X.drop(columns='title')  # Duplicative of purpose

    # Transform sub_grade from "A1" - "G5" to 1.1 - 7.5
    def wrangle_sub_grade(o):
        x = o[1]
        grade = o[0]
        if isinstance(x, float):
            if isinstance(grade, float):
                count_null_subgrade_and_grade += 1
                return float(3)
            return float(ord(grade[0]) - 64)
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10

#     X['sub_grade'] = X['sub_grade'].apply(wrangle_sub_grade)
    X['revol_util'] = X['revol_util'].str.strip('%').astype(float)
    X.sec_app_earliest_cr_line.fillna(False, inplace=True)

    X.earliest_cr_line = X.earliest_cr_line.apply(
        lambda d: datetime.strptime(d, '%b-%Y'))
    X.earliest_cr_line = (X.earliest_cr_line -
                          df_earliest_cr_line_min) / np.timedelta64(1, 'D')
    X.term = X.term.apply(lambda t: int(t[1:3]))
    X['int_rates'] = X.int_rate.apply(lambda r: float(r[:-1]))
    X.drop(['int_rate'], axis=1, inplace=True)
    X['ngrade'] = X[['grade', 'sub_grade']].apply(wrangle_sub_grade, axis=1)
    X.drop(['grade', 'sub_grade'], axis=1, inplace=True)

    def el(e):
        if isinstance(e, float) or e[0] == '<':
            return 0
        return int(e[0:(2 if e[1] == '0' else 1)])
    X.emp_length = X.emp_length.apply(el)

    def wrangle_emp(x):
        if isinstance(x, float):
            return 'No'
        for s in emps:
            if x.find(s) >= 0:
                return s
        return 'Yes'

    def wrangle_list(x, l):
        if isinstance(x, float):
            return False
        for s in l:
            if x.find(s) >= 0:
                return True
        return False

    # Create features for three employee titles: teacher, manager, owner
    X['emp_title'] = X['emp_title'].str.lower()
    X['emp_title_emp'] = X['emp_title'].apply(lambda x: wrangle_emp(x))
    X['emp_title_manager'] = X['emp_title'].apply(
        lambda x: wrangle_list(x, manager))
    X['emp_title_it'] = X['emp_title'].apply(lambda x: wrangle_list(x, it))

    # Drop categoricals with high cardinality
    X = X.drop(columns=['emp_title'])

    # Transform features with many nulls to binary flags
    many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']

    for col in many_nulls:
        try:
            X[col] = X[col].apply(lambda x: 0 if isinstance(x, float) else x)
        except:
            print(col)

    if make_means:
        clist = X.select_dtypes(include=[np.number]).columns.tolist()
        for col in clist:
            means[col] = X[col].mean()

    # For features with few nulls, do mean imputation
    for col in X:
        if X[col].isnull().sum() > 0:
            X[col] = X[col].fillna(means[col])

    print('count_null_subgrade_and_grade', count_null_subgrade_and_grade)
    # Return the wrangled dataframe
    return X

In [200]:
df_train_y =  pd.read_csv('train_labels.csv')

In [201]:
df_train_y.dtypes

id             int64
charged_off    int64
dtype: object

In [202]:
# sns.set(style='ticks', color_codes=True)
# independant_columns = df.columns.drop('y')
# sns.pairplot(data=df,
#                   y_vars=['y'],
#                   x_vars=independant_columns)
# plt.show()

In [203]:
df = wrangle(df, True)

count_null_subgrade_and_grade 0


In [204]:
df.head()

,loan_amnt,funded_amnt,term,installment,emp_length,home_ownership,annual_inc,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,int_rates,ngrade,emp_title_emp,emp_title_manager,emp_title_it
0,12000,12000,36,422.01,4,MORTGAGE,81000.0,debt_consolidation,280xx,NC,12.76,0,20058.0,1,0,0,7,0,4912,23.5,15,w,0,0,Individual,0,0,0,0,174783,1,2,0,4,0,33668,0,2,5,2962,63.0,20900,1,1,2,9,24969.0,0,0,0,0,0,23,1,1,1,0,0,0,0,2,2,2,2,2,9,4,5,2,7,0,0,0,2,92.9,0,0,0,206618,38580,5500,40863,0,False,0,0,0,0,0,0,0,0,0,Cash,16.02,3.5,Yes,False,False
1,6300,6300,36,215.54,4,RENT,39000.0,debt_consolidation,920xx,CA,21.42,0,22340.0,1,0,0,10,0,11876,59.4,12,w,0,0,Individual,0,0,0,0,31046,0,1,0,2,0,19170,0,3,8,5153,72.0,20000,1,0,1,10,3105.0,0,0,0,0,0,72,8,8,0,0,0,0,0,0,6,7,7,8,2,9,10,7,10,0,0,0,3,100.0,0,0,0,43140,31046,16900,23140,0,False,0,0,0,0,0,0,0,0,0,Cash,14.07,3.3,lead,False,False
2,4500,4500,36,139.38,3,RENT,78000.0,debt_consolidation,906xx,CA,2.17,0,18018.0,2,0,0,13,0,1715,5.2,19,w,0,0,Individual,0,0,0,0,35329,0,8,1,2,0,33614,0,1,1,1196,54.0,33300,4,0,3,3,2718.0,0,0,0,0,0,218,10,10,0,0,0,0,0,0,3,3,3,4,13,5,6,3,13,0,0,0,2,100.0,0,0,0,65092,35329,23300,31792,0,False,0,0,0,0,0,0,0,0,0,Cash,7.21,1.3,Yes,False,False
3,12000,12000,36,384.06,10,MORTGAGE,62000.0,debt_consolidation,441xx,OH,13.76,0,19024.0,0,0,0,22,0,9404,16.3,33,w,0,0,Individual,0,0,0,0,75160,4,1,0,2,0,17123,0,6,9,6898,32.0,57600,3,1,2,12,3579.0,0,0,0,0,0,179,1,1,1,0,0,0,0,0,1,3,9,13,7,20,25,3,22,0,0,0,7,100.0,0,0,0,133065,26527,40500,25465,0,False,0,0,0,0,0,0,0,0,0,Cash,9.44,2.1,Yes,False,False
4,12000,12000,36,460.47,10,MORTGAGE,70000.0,debt_consolidation,751xx,TX,14.04,4,18567.0,2,0,0,18,0,17895,24.2,45,w,0,0,Individual,0,0,0,7218,113702,9,2,2,4,0,16711,0,9,10,5373,35.0,73900,3,6,3,14,6317.0,0,0,0,0,0,201,0,0,1,0,0,0,0,6,9,11,11,14,22,15,20,11,18,0,0,2,11,82.2,0,0,0,192842,34606,43300,24499,0,False,0,0,0,0,0,0,0,0,0,Cash,22.35,4.5,Yes,False,False


In [205]:
clist = df.select_dtypes(exclude=[np.number]).columns.tolist()

In [206]:
one_hot_columns = []
binary_columns = []
max_one = 15
for c in  clist:
    if len(df[c].unique()) > max_one:
        binary_columns.append(c)
    else:
        one_hot_columns.append(c)
    

In [33]:
encoders = Pipeline([
                ('binary', ce.BinaryEncoder(cols=binary_columns)),
                ('onehot', ce.OneHotEncoder(use_cat_names=True,cols=one_hot_columns))
#                 ('DecisionTree', DecisionTreeClassifier(max_depth=17, class_weight='balanced'))
            ])
dff = encoders.fit(df)
df_ = dff.transform(df)
# cross_val_score(encoders, df, df_train_y, cv=5, scoring='roc_auc')

In [34]:
X = df_ 
y = df_train_y.charged_off.values

In [35]:
tree = DecisionTreeClassifier(max_depth=5, class_weight='balanced')
pipe = make_pipeline(
    tree
)
cross_val_score(pipe, X, y, cv=5, scoring='roc_auc')

array([0.70341601, 0.70024179, 0.69221389, 0.68881555, 0.69121251])

In [36]:
dfp = tree.fit(X, y)

In [156]:
testdf = pd.read_csv('test_features.csv')

In [157]:
testdf.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method
0,57146,NaN,18500,18500,60 months,25.82%,551.93,E,E4,Account Service Representative,10+ years,MORTGAGE,87000.0,NaN,NaN,debt_consolidation,Debt consolidation,452xx,OH,22.07,0,Jun-2007,3,NaN,111.0,20,1,5504,16.9%,37,w,0,NaN,Individual,NaN,NaN,0,200,212122,2,5,4,6,1.0,105151,88.0,2,3,2991,66.0,32500,6,3,12,10,10606.0,16813.0,23.9,0,0,127.0,55,7,1,2,8.0,NaN,1.0,NaN,0,4,5,8,9,20,14,15,5,20,0.0,0,0,6,100.0,0.0,1,0,250214,110655,22100,113624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
1,20988,NaN,36000,36000,60 months,12.61%,811.95,C,C1,Vice President of IT,7 years,MORTGAGE,165000.0,NaN,NaN,other,Other,480xx,MI,13.39,0,Oct-1992,0,31.0,NaN,12,0,21833,42.6%,21,w,0,NaN,Individual,NaN,NaN,0,0,42059,2,2,1,2,6.0,20226,56.0,1,3,5880,48.0,51200,1,0,1,5,3505.0,19340.0,41.4,0,0,304.0,131,4,4,0,15.0,31.0,6.0,31.0,0,3,7,6,6,6,10,14,7,12,0.0,0,0,2,85.7,16.7,0,0,87584,42059,33000,36384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
2,10031,NaN,12000,12000,36 months,22.90%,463.90,E,E1,Director of Technology,4 years,MORTGAGE,110000.0,NaN,NaN,debt_consolidation,Debt consolidation,972xx,OR,15.82,1,Dec-2004,0,7.0,NaN,4,0,19930,97.2%,11,f,0,NaN,Individual,NaN,NaN,0,0,170912,0,1,1,1,8.0,25739,86.0,0,0,10385,90.0,20500,0,0,0,1,42728.0,570.0,97.2,0,0,130.0,162,42,8,2,42.0,NaN,19.0,NaN,0,2,2,2,3,4,2,5,2,4,0.0,0,0,1,81.8,100.0,0,0,210150,45669,20500,30000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
3,33892,NaN,24875,24875,60 months,11.98%,553.08,B,B5,vp,10+ years,MORTGAGE,105000.0,NaN,NaN,other,Other,463xx,IN,6.91,0,Nov-1995,1,NaN,NaN,7,0,29537,47.3%,24,w,0,NaN,Individual,NaN,NaN,0,0,507349,0,0,0,1,18.0,0,NaN,0,1,12097,47.0,62400,1,1,2,3,84558.0,27863.0,51.5,0,0,126.0,268,22,15,3,65.0,NaN,4.0,NaN,0,4,4,5,7,9,6,12,4,7,0.0,0,0,0,100.0,25.0,0,0,547400,29537,57400,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash
4,35374,NaN,4000,4000,36 months,10.91%,130.79,B,B4,Sales consultant,< 1 year,MORTGAGE,20000.0,NaN,NaN,other,Other,705xx,LA,69.96,0,Sep-2003,0,NaN,NaN,16,0,5863,18.8%,28,w,0,NaN,Joint App,66000.0,32.72,0,0,169662,1,2,1,2,10.0,43585,85.0,0,4,1013,46.0,31200,10,2,5,7,10604.0,4533.0,37.9,0,0,159.0,53,19,2,2,22.0,NaN,4.0,NaN,0,6,10,6,7,12,13,14,10,16,0.0,0,0,2,100.0,0.0,0,0,209852,49448,7300,58652,6169.0,Sep-2003,0.0,2.0,7.0,22.9,3.0,3.0,0.0,0.0,NaN,Cash


In [158]:
testdf_ = wrangle(testdf)

In [159]:
df__ = dff.transform(testdf_)

In [160]:
# testX = imputer.fit_transform(df_) 

In [161]:
py = tree.predict(df__)

In [250]:
with open('submit.csv', 'w') as file:
    file.write('id,charged_off\n')
    for id, charged_off in zip(testdf.id,py):
#         if charged_off > 0:
#             print('a charge off')
        file.write(f"{id},{charged_off}")
        file.write('\n')
    

In [163]:
X_train = df
y_train = y

In [164]:
pca = PCA()
pipe = make_pipeline(
    ce.BinaryEncoder(cols=binary_columns), 
    ce.OneHotEncoder(use_cat_names=True,cols=one_hot_columns),
    RandomForestClassifier(
        n_estimators=100, 
        class_weight='balanced', 
        min_samples_leaf=0.005, 
        oob_score=True, 
        n_jobs=-1)
)

cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc', verbose=10)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ....................... , score=0.7261850427937654, total=   3.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s remaining:    0.0s


[CV] ....................... , score=0.7272992909728996, total=   2.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.9s remaining:    0.0s


...

[CV] ....................... , score=0.7297588099909712, total=   2.2s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   10.6s remaining:    0.0s


[CV] ....................... , score=0.7352598171330698, total=   2.3s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   12.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   12.9s finished


array([0.72618504, 0.72729929, 0.72402357, 0.72975881, 0.73525982])

In [176]:
cp = cross_val_predict(pipe, X_test, cv=5, scoring='roc_auc', verbose=10) 

NameError: name 'pip' is not defined

In [165]:
pf = pipe.fit(X_train,y_train)

In [168]:
py = pipe.predict_proba(testdf_)[:, 1]

In [207]:
# pca = PCA()
pipe = Pipeline(steps = [
    ('be', ce.BinaryEncoder(cols=binary_columns)), 
    ('one', ce.OneHotEncoder(use_cat_names=True,cols=one_hot_columns)),
#     ('pca', pca),
    ('gb', GradientBoostingClassifier())]
)

cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy', verbose=10, n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   15.7s remaining:   23.5s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   15.8s remaining:   10.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   20.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   20.4s finished


array([0.85470199, 0.85481521, 0.85309312, 0.8544178 , 0.854531  ])

array([0.85403974, 0.85388793, 0.853623  , 0.85468274, 0.85426603])

In [110]:
NO_PSUTIL = False
try:
    import psutil
    print('using psutil')
except Exception:
    NO_PSUTIL = True
    print('not using psutil')

using psutil


In [242]:
param_grid = {
#     'pca__n_components': [28],
    "gb__loss" : ['exponential'],
    "gb__learning_rate" : [0.1], 
    "gb__n_estimators": [180],
    "gb__min_samples_leaf": [3],
    "gb__min_impurity_decrease": [1.2],
    "gb__max_depth": [7]
}
# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=2, n_jobs=6, 
                 scoring='roc_auc',
                 verbose=1)

In [243]:
gsf = gs.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   6 out of   8 | elapsed:  1.1min remaining:   22.7s
[Parallel(n_jobs=6)]: Done   8 out of   8 | elapsed:  1.8min finished


In [244]:
print('Best Parameter (roc_auc score=%0.3f):' % gsf.best_score_)
print(gsf.best_params_)

Best Parameter (roc_auc score=0.734):
{'gb__learning_rate': 0.1, 'gb__loss': 'exponential', 'gb__max_depth': 7, 'gb__min_impurity_decrease': 1.2, 'gb__min_samples_leaf': 3, 'gb__n_estimators': 180}


In [245]:
print(count_null_subgrade_and_grade)

0


Best Parameter (roc_auc score=0.734):
{'gb__learning_rate': 0.1, 'gb__loss': 'exponential', 'gb__min_samples_leaf': 3, 'gb__n_estimators': 180}

Best Parameter (CV score=0.734):
{'gb__learning_rate': 0.1, 'gb__loss': 'exponential', 'gb__n_estimators': 150}

In [ ]:

#Plot the PCA spectrum
pca.fit(X)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(pca.explained_variance_ratio_, linewidth=2)
ax0.set_ylabel('PCA explained variance')

print(pca.explained_variance_ratio_)


ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
            linestyle=':', label='n_components chosen')
ax0.legend(prop=dict(size=12))

# For each number of components, find the best classifier results
results = pd.DataFrame(search.cv_results_)
components_col = 'param_pca__n_components'
best_clfs = results.groupby(components_col).apply(
    lambda g: g.nlargest(1, 'mean_test_score'))

In [249]:
py = gsf.predict_proba(testdf_)[:, 1]

In [102]:
a,b,NCm,a*b

(6, 2, 16, 12)